In [24]:
%pip install tf_keras_vis tflite-support-nightly

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tflite-support-nightly (from versions: none)
ERROR: No matching distribution found for tflite-support-nightly


# Create your dataset

1) Choose which classes you would like to add.

2) Create a dataset with your own photos

3) Separate your dataset into train and test subsets.

In [ ]:
from bing_image_downloader import downloader

QUERIES = ["cat"]
OUTPUT_DIR = 'dataset_images'
NB_IMAGES = 60

for query in QUERIES:
    downloader.download(
        query, NB_IMAGES, OUTPUT_DIR,
        adult_filter_off=False, force_replace=False, timeout=15
    )



[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://wallpapercave.com/wp/BzpgP00.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://upload.wikimedia.org/wikipedia/commons/7/75/Cute_grey_kitten.jpg
[%] File Downloaded !

[%] Downloading Image #3 from http://images6.fanpop.com/image/photos/41500000/Cute-Little-Kitten-kittens-41501030-1088-821.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://www.petbarn.com.au/petspot/app/uploads/2019/01/kitten-000017380158_Smaller.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://www.petage.com/wp-content/uploads/2019/09/Depositphotos_74974941_xl-2015-e1569443284386-1024x958.jpg
[%] File Downloaded !

[%] Downloading Image #6 from http://upload.wikimedia.org/wikipedia/commons/9/9b/Photo_of_a_kitten.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://wallup.net/wp-content/uploads/2018/10/06/708179-kittens-kitten-cat-cats-baby-cute-s.jpg
[%] File Download

In [50]:
import os
import pandas as pd
from glob import glob
from sklearn.model_selection import train_test_split


def create_dataframe_from_directories(path):
    data = []

    for label_dir in os.listdir(path):
        if os.path.isfile(label_dir):
            continue

        label_path_path = os.path.join(path, label_dir)
        for file in os.listdir(label_path_path):
            data.append([os.path.join(label_path_path, file), label_dir])
        
    return pd.DataFrame(data, columns = ["path", "label"])

In [58]:
df = create_dataframe_from_directories("dataset_images")
labels = df['label']
df.head()


,path,label
0,dataset_images\bench press\Image_1.gif,bench press
1,dataset_images\bench press\Image_10.jpg,bench press
2,dataset_images\bench press\Image_11.jpeg,bench press
3,dataset_images\bench press\Image_12.jpg,bench press
4,dataset_images\bench press\Image_13.jpg,bench press


array(['Moineau', 'Mésange charbonnière', 'Mésange noire'], dtype=object)

**TODO**: Display some images of your dataset

In [ ]:
import matplotlib.pyplot as plt
import PIL
import PIL.Image
 
plt.figure(figsize=(20, 20))

i = 0
for _, rowdata in df.sample(frac=1).head(9).iterrows():
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(PIL.Image.open(rowdata.path))
    plt.title(rowdata.label)
    plt.axis("off")
    i += 1

In [60]:
trainset, testset, train_labels, test_labels = train_test_split(df, labels, test_size=0.2, random_state=42)

,path,label
139,dataset_images\running\Image_27.png,running
60,dataset_images\deadlift\Image_1.jpg,deadlift
204,dataset_images\squat\Image_31.jpg,squat
186,dataset_images\squat\Image_15.jpg,squat
207,dataset_images\squat\Image_34.jpg,squat
...,...,...
106,dataset_images\deadlift\Image_51.png,deadlift
14,dataset_images\bench press\Image_22.png,bench press
92,dataset_images\deadlift\Image_39.jpg,deadlift
179,dataset_images\running\Image_9.jpg,running


# Explore your dataset

**TODO**: count the number of examples per classes and plot an histogram of it

In [ ]:
import matplotlib.pyplot as plt

# Count the number of examples per class
label_counts = trainset['label'].value_counts()

# Plot the histogram
plt.figure(figsize=(10, 6))
label_counts.plot(kind='bar')
plt.title('Number of examples per class')
plt.xlabel('Class')
plt.ylabel('Number of examples')
plt.xticks(rotation=45)
plt.show()


# Preprocessing steps

In [213]:
from tensorflow.keras.layers import (
    Resizing, Rescaling, RandomFlip, RandomRotation, RandomZoom
)
import tensorflow as tf

from tensorflow.keras import Sequential

IMG_HEIGHT = 224
IMG_WIDTH = 224

image_preprocesses = Sequential([
    Resizing(IMG_HEIGHT, IMG_WIDTH, crop_to_aspect_ratio=True),
    Rescaling(1. / 255)
])

image_augmentations = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomZoom(0.2),
    # TODO: choose some image augmentations layers
    # see which keras layers you can use
    # The kind of layers you will want to use starts with "Random".
    # See the following page for more information on data augmentation :
    # https://www.tensorflow.org/tutorials/images/data_augmentation
])

def open_images(paths):
    return np.stack([
        open_image(path) for path in paths
    ])

def open_image(path):
    with PIL.Image.open(path) as image:
        #opened_img = image_preprocesses(np.asarray(image))
        # Check the number of channels in the image
        if len(np.asarray(image).shape) == 2:  # Grayscale image
            # Duplicate the single channel to create a three-channel image
            image_rgb = np.stack((np.asarray(image),) * 3, axis=-1)
        else:  # Color image
            # Convert the image to RGB if it's not already in RGB mode
            if image.mode != "RGB":
                image = image.convert("RGB")
            # Convert the RGB image to array
            image_rgb = np.asarray(image)
        # Preprocess the image
        preprocessed_img = image_preprocesses(image_rgb)
        print(preprocessed_img.shape)
    return preprocessed_img
    #return opened_img

def augment_images(images):
    return np.stack([
        image_augmentations(img) for img in images
    ])

In [214]:
import matplotlib.pyplot as plt

def open_images(paths):
    return np.stack([open_image(path) for path in paths])

preprocessed_images = open_images(df['path'].tolist())
augmented_images = augment_images(preprocessed_images)

tf.Tensor(
[[[0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  ...
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]]

 [[0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  ...
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]]

 [[0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  ...
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]]

 ...

 [[0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  ...
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]]

 [[0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  [0.6156863 0.6156863 0.6156863]
  ...
  [0.6156863 0.6156863 0.61

In [ ]:
def augment_images(images, num_samples=5):
    augmented_images = np.stack([image_augmentations(img) for img in images])
    return augmented_images[:num_samples]


num_rows = 6
num_cols = 6
total_images = num_rows * num_cols

augmented_images_subset = augment_images(preprocessed_images, num_samples=total_images)


fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 12))
axes = axes.flatten()
for i in range(total_images):
    axes[i].imshow(augmented_images_subset[i])
    axes[i].set_title(f"Augmented Image {i+1}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

**TODO**: plot examples of preprocessed and augmented images

## Define model

In [228]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    GlobalAveragePooling2D,
    Dense,
    Dropout
)

def get_model():
    base_model = MobileNetV2(
        input_shape=(224, 224, 3),
        weights='imagenet',
        include_top=False
    )
    
    layers = base_model.output

    added_layers = [
        # important, we need global average pooling instead of flatten for class activation map later on 
        # Use GlobalAveragePooling2D instead of Flatten for class activation map later on
        GlobalAveragePooling2D(), 
        # Add a dense layer with 128 units and ReLU activation
        Dense(128, activation='relu'),
        # Add the final output layer with softmax activation for classification
        Dense(NUM_CLASSES, activation='softmax')]
        # TODO: ...
    

    model = Model(inputs=base_model.input, outputs=Sequential(added_layers)(layers))

    # TODO: freeze layers
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(
      optimizer = RMSprop(),
      loss = SparseCategoricalCrossentropy(),
      metrics = ['acc']
    )
        
    return model

In [165]:
get_model().summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)   │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer_16[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

# Model training

**TODO**: cross-validate your model, you can base yourself on the code from the first part of this practical work

**NOTE**: you shouldn't augment validations images. What we want to evaluate is the performances on "real" images, not on images that are augmented and change all the time 

# Results

**TODO**: plot the results from your crossvalidation

## Evaluate model on test set

**TODO**: when you've selected a final model, evaluate its performances (f-score, confusion matrix, ...) on your test set

### Display some misclassified images

**TODO**: display some misclassified images to be able to see what images are difficult for the model

## Use grad-cam to inspect how the model classify
Here we can see an heatmap of the regions that activates the model the most.

### For more information:
- tf-keras-vis documentation: https://keisen.github.io/tf-keras-vis-docs/
- Grad-cam paper: https://arxiv.org/pdf/1610.02391.pdf
- Grad-cam++ paper: https://arxiv.org/pdf/1710.11063.pdf

**NOTE**: maybe you will need to adapt some variable names based on your code above. For example, _model_ should be the model trained on all the training data. 

In [41]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from tensorflow.keras.applications import MobileNetV2
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

# Create GradCAM++ object
gradcam = GradcamPlusPlus(
    model,
    model_modifier=ReplaceToLinear(),
    clone=True,
)

nb_images_per_class = 10

fig, axs = plt.subplots(nb_images_per_class, len(LABEL_NAMES), figsize=(len(LABEL_NAMES) * 5, nb_images_per_class * 5))

for label_idx, label_name in enumerate(LABEL_NAMES):
    axs[0,label_idx].set_title(label_name, loc='center', y=1.1)
    
    i = 0
    for _, rowdata in train_df[train_df['label'] == label_name].sample(frac=1).head(nb_images_per_class).iterrows():
        img = open_image(rowdata.path)    
        pred = model(np.expand_dims(img, axis=0))

        # Generate cam with GradCAM++
        score = CategoricalScore(np.argmax(pred))
        cam = gradcam(score, img)
        heatmap = np.uint8(cm.jet(cam[0])[..., :3] * 255)

        axs[i,label_idx].imshow(img, vmin=0, vmax=1)
        axs[i,label_idx].imshow(heatmap, cmap='jet', alpha=0.5) # overlay
        axs[i,label_idx].axis("off")

        i += 1

NameError: name 'model' is not defined

### Grad-cam on misclassified images

**TODO**: now do the same than above, but with the misclassified images. You shouldn't change anything pertaining to grad-cam besides on which image and prediction you do it

**TODO**: try to determine what caused the misclassifcations and what you can do about it